# Day 8: Resonant Collinearity

## Import libraries

In [6]:
import copy
from collections import defaultdict


## Import data

In [13]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - The grid map represents antennas.
# - EACH antenna is represented by a *letter* or *digit*.
# - The FREQUENCY of EACH antenna is different, based on whether the antenna is a: lowercase letter, uppercase letter or digit.
# - '#': Represents an antinode. 
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-8_input-test.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

# Separate data by line to create rows for grid
grid = file_data.split("\n")

# Separate data in EACH row to represent EACH column
for i in range(len(grid)):
    grid[i] = list(grid[i])

print(grid)
# ====================================================================================================================

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'], ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'], ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]


## Helper functions

Total unique locations containing an antinode: 0


## Part 1

In [15]:
# *** [PART 1] ***
# ! PROBLEM: The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas.
# - In particular, an *antinode* occurs at *any point* that is perfectly *in line* with TWO antennas of the SAME frequency - but ONLY when ONE of the antennas is TWICE as far away as the other. This means that for any PAIR of antennas with the SAME frequency, there are TWO antinodes, one on EITHER side of them.
# - Antennas with DIFFERENT frequencies DO NOT create antinodes; however, antinodes CAN occur at the SAME locations that CONTAIN antennas.
# - TODO: Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?
# ====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part1_grid = copy.deepcopy(grid)
antennas = []
numRows = len(grid)
numCols = len(grid[0])
antinodes = set()
freq_map = defaultdict(list)

# Store antennas & their respective locations
for x in range(numRows):
    for y in range(numCols):
        if grid[x][y] != '.':
            antennas.append((grid[x][y], x, y)) # ('A', 4, 2)

""" Calculate the positions of antinodes """
# Group antennas by frequency
for ant_freq, x, y in antennas:
    freq_map[ant_freq].append((x, y))

# Calculate antinodes for each frequency
for ant_freq, positions in freq_map.items():
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)): # Compare every single antenna against every other antenna of the same frequency
            x1, y1 = positions[i]
            x2, y2 = positions[j]
            
            # Calculate the midpoint between the 2 antenna points
            mid_x = (x1 + x2) / 2
            mid_y = (y1 + y2) / 2
            
            # Check if one antenna is twice as far as the other
            # - Calculate the distance between antenna point #1 & point #2
            # - IF both points = the samem then know that those 2 antennas are twice as far away from one another 
            dx = x2 - x1
            dy = y2 - y1

            if dx == 0:
                if abs(y2 - y1) == 2 * abs(y1 - mid_y):
                    antinodes.add((int(mid_x), int(mid_y)))
            elif dy == 0:
                if abs(x2 - x1) == 2 * abs(x1 - mid_x):
                    antinodes.add((int(mid_x), int(mid_y)))
            else:
                if abs(x2 - x1) == 2 * abs(x1 - mid_x) and abs(y2 - y1) == 2 * abs(y1 - mid_y):
                    antinodes.add((int(mid_x), int(mid_y)))

print(len(antinodes))
# ====================================================================================================================

8


## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: xxx
# - TODO: xxx
#====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part2_grid = copy.deepcopy(grid)
